# Error Projections

In [ ]:
#  Plotly Visualization
import plotly.graph_objects as go
import plotly.express       as px
import plotly.subplots      as sp

# OpenCV
import cv2

# Pandas 
import pandas as pd

import math

#  Numpy
import numpy as np

import scipy.interpolate

from pyproj import CRS, Transformer

# Terminus APIs
from tmns.core.types import GCP
from tmns.dem.gtiff import DEM_File as DEM
from tmns.dem.fixed import Fixed_DEM as Flat
from tmns.proj.RPC00B import RPC00B
from tmns.proj.SENSRB import SENSRB

In [ ]:
srtm_path = 'SRTM_GL1.tif'
dem = DEM( srtm_path )

In [ ]:
img_path = 'ARUDEN000040045.2.tif'

In [ ]:
img_bands = cv2.cvtColor( cv2.imread( img_path, cv2.IMREAD_COLOR ), cv2.COLOR_BGR2RGB )
print(img_bands.shape)

In [ ]:
fig1 = go.Figure()
fig1.add_trace( go.Image( z = img_bands ) )
fig1.update_layout( title = 'USGS Aerial Photo, 4/23/2002',
                    height = 700 )
fig1.show( renderer = 'png' )

## Setup Ground Control Points

In [ ]:
gcp_df = pd.read_csv( 'GCPs.csv' )
mean_elevation = gcp_df['Elevation'].mean()
gcp_df['Elevation'] = mean_elevation
display( gcp_df.head(5) )
display( f'Total of {gcp_df.shape[0]} GCPs loaded' )

In [ ]:
crd_xform = Transformer.from_crs( 4326, 32613 )

In [ ]:
lla_gcp_list = []
utm_gcp_list = []
counter = 0
for gcp in gcp_df.itertuples():

    utm = crd_xform.transform( gcp.Latitude, gcp.Longitude )

    lla_gcp_list.append( GCP( id = counter,
                              pixel = np.array( [ gcp.PX, gcp.PY ], dtype = np.float64 ),
                              coordinate = np.array( [ gcp.Longitude, gcp.Latitude, gcp.Elevation ], dtype = np.float64 ) ) )
    
    utm_gcp_list.append( GCP( id = counter,
                              pixel = np.array( [ gcp.PX, gcp.PY ], dtype = np.float64 ),
                              coordinate = np.array( [ utm[0], utm[1], gcp.Elevation ], dtype = np.float64 ) ) )
    counter += 1

![GCP List](./docs/gcp_list.png)

## 4 Corner Comparison

The image from the USGS attempts to define 4 corners.  The equation is given by the following:

$$
t_x = 
$$

$$
X_\texttt{top} = \lambda_{tl) \cdot (1-t) + \lambda_tr \cdot  t
$$

In [ ]:
model = { 'tl': np.array( crd_xform.transform( 39.765563, -104.972699 ) ),
          'tr': np.array( crd_xform.transform( 39.765335, -104.932675 ) ),
          'bl': np.array( crd_xform.transform( 39.734676, -104.972985 ) ),
          'br': np.array( crd_xform.transform( 39.734448, -104.932979 ) ) }

points = []
deltas = []
sumD = 0
for gcp in utm_gcp_list:

    tx = gcp.pixel[0] / img_bands.shape[0]
    ty = gcp.pixel[1] / img_bands.shape[1]

    p1 = model['tl'] * (1-tx) + model['tr'] * tx
    p2 = model['bl'] * (1-tx) + model['br'] * tx

    pt = p1 * (1-ty) + p2 * ty
    points.append( [float(x) for x in pt] )

    delta = pt - gcp.coordinate[0:2]
    deltas.append( math.sqrt( np.dot( delta, delta ) ) )
    sumD += deltas[-1]

rms_corners = math.sqrt( sumD / len(utm_gcp_list))
min_x = min( model['tl'][0], model['bl'][0] )
max_x = max( model['tr'][0], model['br'][0] )
min_y = min( model['tl'][1], model['bl'][1] )
max_y = max( model['tr'][1], model['br'][1] )

grid_x, grid_y = np.mgrid[ min_x:max_x:50, min_y:max_y:50]
grid_vals = scipy.interpolate.griddata( points, deltas, (grid_x, grid_y), method='linear' )


In [ ]:
fig2 = px.imshow( grid_vals )
fig2.update_layout( height = 600, width = 800 )
fig2.write_image( 'docs/4-corner-diffs.png' )
fig2.show( renderer = 'png' )

In [ ]:
from IPython.display import Image, HTML, display

image_A = './docs/gcp_list.png'
image_B = './docs/4-corner-diffs2.png'

display(HTML("<table><tr><td><img width=400 src={0}></td><td><img width=400 src={1}></td></tr></table>".format(image_A,image_B)))

In [ ]:
print( f'RMS Error: {rms_corners}' )

## Geo-Transform Comparison



In [ ]:
image_size = np.array( [2829.0, 2964.0], dtype = np.float64 )

#  Create model from GCPs
new_model = SENSRB.solve( gcps       = utm_gcp_list,
                          image_size = image_size )
        
#  Verify the model
sumDelta = 0
points = []
deltas = []
minVal = np.copy(utm_gcp_list[0].coordinate)[0:2]
maxVal = np.copy(utm_gcp_list[0].coordinate)[0:2]
maxDelta = 0
for gcp in utm_gcp_list:

    #  World coordinate
    utm = new_model.pixel_to_world( gcp.pixel,
                                    coord_epsg = 32613,
                                    dem_model = dem )

    delta = utm - gcp.coordinate
    sumDelta += math.sqrt( np.dot( delta, delta ) )

    points.append( utm[0:2] )
    deltas.append( math.sqrt( np.dot( delta, delta ) ) )

    minVal[0] = min( minVal[0], gcp.coordinate[0] )
    maxVal[0] = max( maxVal[0], gcp.coordinate[0] )
    minVal[1] = min( minVal[1], gcp.coordinate[1] )
    maxVal[1] = max( maxVal[1], gcp.coordinate[1] )

    maxDelta = max( maxDelta, math.sqrt( np.dot( delta, delta ) ) )

rms_geotransform = math.sqrt( sumDelta / len(utm_gcp_list) )
print( f'RMS Error: {rms_geotransform}, Max Delta: {maxDelta}' )

grid_x, grid_y = np.mgrid[ minVal[0]:maxVal[0]:50, minVal[1]:maxVal[1]:50]
grid_vals = scipy.interpolate.griddata( points, deltas, (grid_x, grid_y), method='linear' )

In [ ]:
fig3 = px.imshow( grid_vals )
fig3.update_layout( height = 600, width = 800 )
fig3.write_image( 'docs/geotransform-diffs.png' )
fig3.show( renderer = 'png' )

In [ ]:
from IPython.display import Image, HTML, display

image_A = './docs/gcp_list.png'
image_B = './docs/geotransform-diffs2.png'

display(HTML("<table><tr><td><img width=400 src={0}></td><td><img width=400 src={1}></td></tr></table>".format(image_A,image_B)))

## Computing RPCs


In [ ]:
flat_model = Flat(1612)

#  Create model from GCPs
new_model = RPC00B.solve( gcps       = lla_gcp_list,
                          dem        = flat_model,
                          image_size = image_size,
                          method     = 'B' )
        
#  Verify the model
sumDelta = 0
points = []
deltas = []
minVal = np.copy(lla_gcp_list[0].coordinate)[0:2]
maxVal = np.copy(lla_gcp_list[0].coordinate)[0:2]
maxDelta = 0

for gcp in lla_gcp_list:

    #  World coordinate
    #lla = new_model.pixel_to_world( gcp.pixel,
    #                                dem_model  = dem,
    #                                method     = 'B' )

    #print( f'lla: {lla}, gcp: {gcp.coordinate}' )
    #delta = lla - gcp.coordinate
    pix = new_model.world_to_pixel( gcp.coordinate,
                                    method = 'B' )
    delta = pix - gcp.pixel
    
    sumDelta += math.sqrt( np.dot( delta, delta ) )

    points.append( utm[0:2] )
    deltas.append( math.sqrt( np.dot( delta, delta ) ) )

    minVal[0] = min( minVal[0], gcp.coordinate[0] )
    maxVal[0] = max( maxVal[0], gcp.coordinate[0] )
    minVal[1] = min( minVal[1], gcp.coordinate[1] )
    maxVal[1] = max( maxVal[1], gcp.coordinate[1] )

    maxDelta = max( maxDelta, math.sqrt( np.dot( delta, delta ) ) )

rms_geotransform = math.sqrt( sumDelta / len(lla_gcp_list) )
print( f'RMS Error: {rms_geotransform}, Max Delta: {maxDelta}' )

print( f'minVal: {minVal}, maxVal: {maxVal}' )
grid_x, grid_y = np.mgrid[ minVal[0]:maxVal[0]:0.001, minVal[1]:maxVal[1]:0.001]
grid_vals = scipy.interpolate.griddata( points, deltas, (grid_x, grid_y), method='linear' )